In [210]:
import polars as pl
import json

with open('./sysmon-3.json') as f:
  src_json = json.loads(f.read())
src = pl.from_dicts(src_json)
  
with open('./super-nested.json') as f:
  super_nested_json = json.loads(f.read())
super_nested = pl.from_dicts(super_nested_json)

In [211]:
src

_index,_type,_id,_score,metadata,agent,process,winlog,log,message,tags,observer,@timestamp,ecs,host,@version,event,user
str,str,str,f64,struct[4],struct[6],struct[8],struct[13],struct[1],str,list[str],struct[1],str,struct[1],struct[7],str,struct[8],struct[1]
"""so-beats-2022.10.21""","""_doc""","""p5M9-4MBqn-zbgNi1QBP""",12.011028,"{""winlogbeat"",""192.168.125.10"",""_doc"",""7.17.1""}","{""asarea"",""asarea"",""6637b2dd-689f-4ab6-9581-34a1268d4a6d"",""cf4a3abc-4d74-4a05-affa-dd08e5306cb2"",""winlogbeat"",""7.17.1""}","{{""{93591ace-ae34-6348-1000-000000000700}"",""C:\Windows\system32\svchost.exe -k DcomLaunch -p"",""C:\Windows\System32\svchost.exe""},{""10.0.18362.1 (WinBuild.160101.0800)"",""Microsoft® Windows® Operating System"",""WMI Provider Host"",""Microsoft Corporation"",""Wmiprvse.exe""},""4856"",""C:\Windows\system32\"",""{93591ace-bff2-6352-4e19-000000000700}"",""C:\Windows\System32\wbem\WmiPrvSE.exe"",""C:\Windows\system32\wbem\wmiprvse.exe -secured -Embedding"",""808""}","{""asarea.vxnwua.net"",{4176,{7140}},""Microsoft-Windows-Sysmon/Operational"",{""{93591ace-ae35-6348-e403-000000000000}"",""0"",""System"",""NT AUTHORITY\SYSTEM"",""2022-10-21 15:51:14.451"",""MD5=801E8003C257C8F540B20F1E0DECD3A6"",""0x3e4"",""-""},""Info"",5,6386012,""Process Create (rule: ProcessCreate)"",""1"",""{5770385f-c22a-43e0-bf4c-06f5698ffbd9}"",""wineventlog"",""Microsoft-Windows-Sysmon"",{""S-1-5-18"",""NT AUTHORITY"",""SYSTEM"",""User""}}","{""information""}","""Process Create: RuleName: - Ut…","[""beat-ext"", ""beats_input_codec_plain_applied""]","{""asarea.vxnwua.net""}","""2022-10-21T15:51:14.476Z""","{""1.12.0""}","{""asarea"",{""18362.239"",""10.0.18362.239 (WinBuild.160101.0800)"",""Windows 10 Pro"",""windows"",""windows"",""10.0"",""windows""},[""fe80::112c:2008:42d7:6c99"", ""192.168.125.10""],""asarea.vxnwua.net"",""93591ace-8aa7-449c-a6ea-adbb095693e4"",[""e6:71:7f:aa:77:35""],""x86_64""}","""1""","{""1"",""Microsoft-Windows-Sysmon"",""2022-10-21T15:51:15.633Z"",""event"",""sysmon"",""Process Create (rule: ProcessCreate)"",""host,process"",""process_creation""}","{""NT AUTHORITY\NETWORK SERVICE""}"
"""so-beats-2022.10.21""","""_doc""","""_pM9-4MBqn-zbgNi1QBi""",12.011028,"{""winlogbeat"",""192.168.125.10"",""_doc"",""7.17.1""}","{""asarea"",""asarea"",""6637b2dd-689f-4ab6-9581-34a1268d4a6d"",""cf4a3abc-4d74-4a05-affa-dd08e5306cb2"",""winlogbeat"",""7.17.1""}","{{""{93591ace-ae33-6348-0b00-000000000700}"",""C:\Windows\system32\services.exe"",""C:\Windows\System32\services.exe""},{""10.0.18362.239 (WinBuild.160101.0800)"",""Microsoft® Windows® Operating System"",""Microsoft Software Protection Platform Service"",""Microsoft Corporation"",""sppsvc.exe""},""1828"",""C:\Windows"",""{93591ace-bff3-6352-4f19-000000000700}"",""C:\Windows\System32\sppsvc.exe"",""C:\Windows\system32\sppsvc.exe"",""612""}","{""asarea.vxnwua.net"",{4176,{7140}},""Microsoft-Windows-Sysmon/Operational"",{""{93591ace-ae35-6348-e403-000000000000}"",""0"",""System"",""NT AUTHORITY\SYSTEM"",""2022-10-21 15:51:15.015"",""MD5=AAB634FA7C0EEEE6EE64C138A5FDBC89"",""0x3e4"",""-""},""Info"",5,6386151,""Process Create (rule: ProcessCreate)"",""1"",""{5770385f-c22a-43e0-bf4c-06f5698ffbd9}"",""wineventlog"",""Microsoft-Windows-Sysmon"",{""S-1-5-18"",""NT AUTHORITY"",""SYSTEM"",""User""}}","{""information""}","""Process Create: RuleName: - Ut…","[""beat-ext"", ""beats_input_codec_plain_applied""]","{""asarea.vxnwua.net""}","""2022-10-21T15:51:15.050Z""","{""1.12.0""}","{""asarea"",{""18362.239"",""10.0.18362.239 (WinBuild.160101.0800)"",""Windows 10 Pro"",""windows"",""windows"",""10.0"",""windows""},[""fe80::112c:2008:42d7:6c99"", ""192.168.125.10""],""asarea.vxnwua.net"",""93591ace-8aa7-449c-a6ea-adbb095693e4"",[""e6:71:7f:aa:77:35""],""x86_64""}","""1""","{""1"",""Microsoft-Windows-Sysmon"",""2022-10-21T15:51:15.919Z"",""event"",""sysmon"",""Process Create (rule: ProcessCreate)"",""host,process"",""process_creation""}","{""NT AUTHORITY\NETWORK SERVICE""}"
"""so-bea

## Goal
Extract the following fields while maintaining structure:
```
@timestamp, host.name, host.os.name, process.command_line
```

## Accessing an column of strings

In [244]:
src.select('@timestamp')

@timestamp
str
"""2022-10-21T15:51:14.476Z"""
"""2022-10-21T15:51:15.050Z"""
"""2022-10-21T15:51:11.115Z"""


## Accessing column of objects

In [246]:
# Extracts a column of structs, then expands the struct into another frame
host = src.select('host')
# host['host'].dtype == Struct
host = host.unnest('host')
host

hostname,os,ip,name,id,mac,architecture
str,struct[7],list[str],str,str,list[str],str
"""asarea""","{""18362.239"",""10.0.18362.239 (WinBuild.160101.0800)"",""Windows 10 Pro"",""windows"",""windows"",""10.0"",""windows""}","[""fe80::112c:2008:42d7:6c99"", ""192.168.125.10""]","""asarea.vxnwua.net""","""93591ace-8aa7-449c-a6ea-adbb09…","[""e6:71:7f:aa:77:35""]","""x86_64"""
"""asarea""","{""18362.239"",""10.0.18362.239 (WinBuild.160101.0800)"",""Windows 10 Pro"",""windows"",""windows"",""10.0"",""windows""}","[""fe80::112c:2008:42d7:6c99"", ""192.168.125.10""]","""asarea.vxnwua.net""","""93591ace-8aa7-449c-a6ea-adbb09…","[""e6:71:7f:aa:77:35""]","""x86_64"""
"""asarea""","{""18362.239"",""10.0.18362.239 (WinBuild.160101.0800)"",""Windows 10 Pro"",""windows"",""windows"",""10.0"",""windows""}","[""fe80::112c:2008:42d7:6c99"", ""192.168.125.10""]","""asarea.vxnwua.net""","""93591ace-8aa7-449c-a6ea-adbb09…","[""e6:71:7f:aa:77:35""]","""x86_64"""


In [248]:
# Extracts a column of structs, then expands the struct into another frame
os = host.select('os')
# host['host'].dtype == Struct
os = os.unnest('os')

In [249]:
# Extracts a column of strings from the above
name = os.select('name')
# name['name'].dtype == String
name

name
str
"""Windows 10 Pro"""
"""Windows 10 Pro"""
"""Windows 10 Pro"""


## Rebuild structured data from the above

In [216]:
name_struct = pl.DataFrame(name)
name_struct


name
str
"""Windows 10 Pro"""
"""Windows 10 Pro"""
"""Windows 10 Pro"""


In [217]:
os_struct = pl.DataFrame({'os': name_struct.to_struct()})
os_struct

os
struct[1]
"{""Windows 10 Pro""}"
"{""Windows 10 Pro""}"
"{""Windows 10 Pro""}"


In [218]:
host_struct = pl.DataFrame({'host': os_struct.to_struct()})
host_struct

host
struct[1]
"{{""Windows 10 Pro""}}"
"{{""Windows 10 Pro""}}"
"{{""Windows 10 Pro""}}"


In [219]:
# Grab it back out
host_struct.to_dicts()

[{'host': {'os': {'name': 'Windows 10 Pro'}}},
 {'host': {'os': {'name': 'Windows 10 Pro'}}},
 {'host': {'os': {'name': 'Windows 10 Pro'}}}]

## More recursive example

In [274]:
field = 'layer1.layer2.layer3.layer4.layer5.layer6.layer7.layer8'

def recurse(data:pl.DataFrame, fields:list[str], index:int=0):
    if index == len(fields):
        return pl.DataFrame({fields[index-1]: data.to_struct()})
    
    new = data.select(fields[index]).unnest(fields[index])
    newi = index + 1
    
    rec_data = recurse(new, fields, newi)
        
    if index == 0:
        return rec_data
    else:
        return pl.DataFrame({fields[index-1]: rec_data.to_struct()})
    
recurse(super_nested, field.split('.')).to_dicts()

[{'layer1': {'layer2': {'layer3': {'layer4': {'layer5': {'layer6': {'layer7': {'layer8': {'layer9': {'layer10': 'you win!'}}}}}}}}}},
 {'layer1': {'layer2': {'layer3': {'layer4': {'layer5': {'layer6': {'layer7': {'layer8': {'layer9': {'layer10': 'you win!'}}}}}}}}}},
 {'layer1': {'layer2': {'layer3': {'layer4': {'layer5': {'layer6': {'layer7': {'layer8': {'layer9': {'layer10': 'you win!'}}}}}}}}}},
 {'layer1': {'layer2': {'layer3': {'layer4': {'layer5': {'layer6': {'layer7': {'layer8': {'layer9': {'layer10': 'you win!'}}}}}}}}}},
 {'layer1': {'layer2': {'layer3': {'layer4': {'layer5': {'layer6': {'layer7': {'layer8': {'layer9': {'layer10': 'you win!'}}}}}}}}}}]

## Better

In [289]:
def get_element(data:pl.DataFrame, fields:list[str], index:int=0):
    if index == len(fields):
        return pl.DataFrame({fields[index-1]: data.to_struct()})
    
    split = fields[index]
  
    new = data.select(split)
    
    if len(fields) == 1:
        return new
    
    # print(new.to_dicts())
    
    if isinstance(new[split].dtype, pl.Struct):
        new = new.unnest(split)
    else:
        return pl.DataFrame({fields[index-1]: new.to_struct()})
        
    rec_data = get_element(new, fields, index + 1)
    
    if index == 0:
        return rec_data
    else:
        return pl.DataFrame({fields[index-1]: rec_data.to_struct()})

field = '@timestamp'
print(get_element(src, field.split('.')).to_dicts())

field = 'host.name'
print(get_element(src, field.split('.')).to_dicts())

field = 'host.os.name'
print(get_element(src, field.split('.')).to_dicts())

field = 'process.command_line'
print(get_element(src, field.split('.')).to_dicts())


[{'@timestamp': '2022-10-21T15:51:14.476Z'}, {'@timestamp': '2022-10-21T15:51:15.050Z'}, {'@timestamp': '2022-10-21T15:51:11.115Z'}]
[{'host': {'name': 'asarea.vxnwua.net'}}, {'host': {'name': 'asarea.vxnwua.net'}}, {'host': {'name': 'asarea.vxnwua.net'}}]
[{'host': {'os': {'name': 'Windows 10 Pro'}}}, {'host': {'os': {'name': 'Windows 10 Pro'}}}, {'host': {'os': {'name': 'Windows 10 Pro'}}}]
[{'process': {'command_line': 'C:\\Windows\\system32\\wbem\\wmiprvse.exe -secured -Embedding'}}, {'process': {'command_line': 'C:\\Windows\\system32\\sppsvc.exe'}}, {'process': {'command_line': 'taskhostw.exe'}}]


# Merging

In [321]:
def advance(columns:list[pl.DataFrame]) -> list[pl.DataFrame]:
    new = []
    name = columns[0].columns[0]
    for i in columns:
        new.append(i.select(name).unnest(name))
        
    return new

def merge(frames):
    columns = {}
    for i in frames:
        name = i.columns[0]
        
        if name not in columns:
            columns[name] = []
        
        columns[name].append(i)
        
    mergable = []
    for i in columns:
        if len(columns[i]) == 1:
            mergable.append(columns[i][0])
            continue

        new = pl.DataFrame({i: merge(advance(columns[i])).to_struct()})

        mergable.append(new)
        
    return pl.concat(mergable, how="horizontal")

fields = "@timestamp, host.name, host.os.name, process.command_line".split(', ')

frames = []
for idx, field in enumerate(fields):
    splits = field.split('.')
    frames.append(get_element(src, splits))

merge(frames).to_dicts()

[{'@timestamp': '2022-10-21T15:51:14.476Z',
  'host': {'name': 'asarea.vxnwua.net', 'os': {'name': 'Windows 10 Pro'}},
  'process': {'command_line': 'C:\\Windows\\system32\\wbem\\wmiprvse.exe -secured -Embedding'}},
 {'@timestamp': '2022-10-21T15:51:15.050Z',
  'host': {'name': 'asarea.vxnwua.net', 'os': {'name': 'Windows 10 Pro'}},
  'process': {'command_line': 'C:\\Windows\\system32\\sppsvc.exe'}},
 {'@timestamp': '2022-10-21T15:51:11.115Z',
  'host': {'name': 'asarea.vxnwua.net', 'os': {'name': 'Windows 10 Pro'}},
  'process': {'command_line': 'taskhostw.exe'}}]